In [37]:
import torch
import matplotlib.pyplot as plt
from utils.defects import S_cal
import numpy as np

from kinetic_solver import KineticSolver, KineticData

def _action2light(self, action, grid_size=256):

    x_center = int((action[0]+1)/2 * grid_size)   # 椭圆中心x坐标
    y_center = int((action[1]+1)/2 * grid_size)   # 椭圆中心y坐标
    a = (action[2]+1)/2 * grid_size / 2                # 长轴
    b = (action[3]+1)/2 * grid_size / 2                # 短轴
    theta = (action[4]+1)/2 * 2 * np.pi           # 方向角范围 [0, 2π]
    intensity = (action[5]+1)/2 * 4 + 1           # 光照强度范围 [1, 10]

    y, x = np.ogrid[:grid_size, :grid_size]
    
    # 3. 旋转椭圆的角度变换
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # 将坐标平移到椭圆中心
    x_shifted = x - x_center
    y_shifted = y - y_center
    
    # 旋转后的坐标
    x_rotated = x_shifted * cos_theta + y_shifted * sin_theta
    y_rotated = -x_shifted * sin_theta + y_shifted * cos_theta
    
    # 4. 椭圆方程 (x_rotated / a)^2 + (y_rotated / b)^2 <= 1 用于定义椭圆内的区域
    ellipse_mask = (x_rotated / (a + 1e-9))**2 + (y_rotated / (b+ 1e-9))**2 <= 1

    # 5. 创建光照矩阵，初始值都设为1，表示椭圆外部区域
    light_matrix = np.ones((grid_size, grid_size))

    # 6. 渐变效果 - 根据椭圆边缘距离生成，椭圆内从1到1.5渐变
    distance = np.sqrt((x_rotated / (a + 1e-6))**2 + (y_rotated / (b+ 1e-6))**2)
    light_matrix = np.where(ellipse_mask, intensity - (intensity-1) * distance, light_matrix)

    return light_matrix, ellipse_mask

In [ ]:
action = (0.5, 0, -0.5, 0.1, 0.1, 1)
light_mat, ellipse_mask = _action2light(None, action, grid_size=256)
import matplotlib.pyplot as plt
plt.imshow(ellipse_mask)
plt.colorbar()
plt.show()

In [ ]:
import numpy as np
d11, d12 = load_data.get_D()
S = S_cal(d11.cpu().data.numpy(), d12.cpu().data.numpy())
print(S.mean())
d11 = d11.cpu().data.numpy()
d12 = d12.cpu().data.numpy()
theta = 0.5 * np.arctan2(2*d12, 2*d11-1)
# plot S

cost, sint = np.cos(theta), np.sin(theta)

fig, ax1 = plt.subplots(1,3, figsize=(15, 5))
ax1[0].streamplot(np.arange(0, theta.shape[0]), np.arange(0, theta.shape[1]),
                cost, sint, arrowsize=0.5, color='r',
                density=4, linewidth=0.5)

ax1[1].imshow(d11)
# set colorbar

ax1[2].imshow(S)
plt.show()
